# tsma 2: How to visualize one simulation ?
All the examples are done with an implementation of Gross's model of 2022: https://doi.org/10.1016/j.ecolecon.2010.03.021

### requirements

In [1]:
from tsma.models.gross2022 import Gross2022 as model

parameters = {
    "b_margin": 0.05,
    "hh_cons_propensity": 1,
    "norisk_interest": 0,
    "tgt_capital_ratio": 0.1,
    "smooth_interest": 15,
    "beta": 1, 
}

hyper_parameters = {"f_n": 200, "hh_n": 20000, "t_end": 100, "seed": 33}

initial_values = {
    "wages_0": 1,
    "wages_1": 1,
    "prices_0": 1.01,
    "prices_1": 1,
}

m = model(parameters, hyper_parameters, initial_values)
output = m.simulate(t_end = 40, seed = 10, save = True, overwrite = False)

## 1. The figures

All the figures are defined in **visuals.figures**

### Time series mosaic

In **mosaique**, **mosaique_hist**, the time series are plotted in separated plots.

In [3]:
from tsma.visuals.figures import mosaique, mosaique_hist

fig1 = mosaique(output, ncol = 4, title = "Test 1 sim")
fig1.show()

fig2 = mosaique_hist(output, ncol = 5, title ="Test 1 sim hist", pal = "viridis")
fig2.show()

### Adapted mosaic

A cleaner visualization uses a dictionary **dct_groups** to gather the curves and give the titles wanted.

In [4]:
dct_groups = {
    "Active Firms": ["n_active"],
    "Nominal GDP Growth": ["GDP_growth", "unemployment_rate"],
    "Debt to GDP": ["Debt_to_GDP"],
    "Nominal loan Growth": ["loans_growth"],
    "Wage Inflation at Firms": ["mean_wage_inflation"],
    "Price Inflation at Firms": ["mean_inflation"],
    "Firm Profit share in GDP": ["f_profit_share_GDP"],
    "Wage Share in GDP": ["w_share_GDP"],
    "Firm's Interest Coverage": ["firms_interest_coverage"],
    "Firms Default Rate": ["default_rate"],
    "Loss Given Default": ["sLGD"],
    "Loan Interest rate": ["interest"],
    "Full debt service and Rollover": ["full_debt_service", "rollover"],
    "New Loans and Top-up": ["count_newloans", "count_topups"],
    "Firms' and Banks' Capital Ratios": ["capital_ratio", "mean_f_capr"],
    # ----
    "Inflation": ["inflation"],
    "Wage Inflation": ["wage_inflation"],
    "Real Wage": ["real_wage"],
    "GDP": ["GDP"],
    "Real GDP": ["real_GDP"],
    "Real GDP Growth": ["real_GDP_growth"],
    "Loans and interest expenses": ["loans", "interest_expenses"],
    "Default proba": ["sPD"],
    "Banks' networth and losses": ["b_lossnet", "b_networth"],
    "Dividends and HH cash": ["dividends", "sum_hh_cash"],
    "HH earnings and savings": ["mean_wage", "mean_hh_cash"],
    "Demande and Price level": ["mean_price", "mean_demand"],
    "Firms Payements and Cash": ["sum_wage", "sum_f_cash"],
    "Firms cash and loan": ["mean_loan", "mean_f_cash"],
    "std earnings and savings": ["std_wage", "std_hh_cash"],
    "std Demande and Price": ["std_price", "std_demand"],
    "std Firms Cash and Loan": ["std_loan", "std_f_cash"],
}

**auto_dct_groups**(varnames, agents_names) can be used to automatically create this dictionary based on agents code.
This can be useful for ABMs when multiple agents of similar types are involved.

In [5]:
from tsma.basics import auto_dct_groups

auto_dct_groups(["state_a", "state_b", "transfert_a_b", "transfert_a_c"], ["a", "b", "c"])

{'transfert a': ['transfert_a_b', 'transfert_a_c'],
 'state': ['state_a', 'state_b']}

Finally, Plotly accepts certain **html** commands. So let's improve the title: 

In [7]:
from tsma.visuals.figures import adapted_mosaique

title = """<a href = "https://doi.org/10.1016/j.ecolecon.2010.03.021" >Gross 2022 </a>"""

fig = adapted_mosaique(
    output,
    title,
    dct_groups,
)
fig.show()

### Custom Adapted mosaique

To change the appearance, change :
- **f_line**  for the lines' style and color 
- **yaxis_to_right** for the variables's yaxis to put to the right
- **ncol, nskip**  for the number of columns and blanck blocks in the beginning
- **width, height, hspace, vspace** for the dimensions

In [ ]:
import seaborn as sns 

title = """
<a href = "https://doi.org/10.1016/j.ecolecon.2010.03.021" >Gross 2022</a> 
<br> 
<br> <b>Description</b> :  
<br> The space is given by <i>nskip</i>
"""

def new_f_line(varname, k):
    type_line = "solid" 
    color = sns.color_palette("viridis", 3).as_hex()[k]
    if "tgt" in varname.split("_"):
        type_line = "dot"
    opa = 0.9    
    width = 2
    return type_line, color, opa, width

fig = adapted_mosaique(
    output,
    title,
    dct_groups,
    
    f_line = new_f_line,
    yaxis_to_right = ["rollover", "unemployment_rate", "interest_expenses"],
    
    ncol = 4,
    nskip = 2,
    
    height = 1600,
    width = 1400
)
fig.show()

## 2 figures saving

To **save** plotly figures use :
- **get_save_path**(model, prev_folder: bool = False) -> str: 
- **save_fig**(fig, newpath_fig: str, figname: str, save_format: str = "html") -> None

In [8]:
import os
from tsma.collect.output_management import  get_save_path 
from tsma.visuals.fig_management import save_fig

path_figure = os.sep.join([get_save_path(m, False), "figures"])
save_fig(fig, newpath_fig = path_figure, figname = "test_save", save_format = "png")
print(path_figure)

C:\Users\samud\Bureau\Python code\Repo\tsma\Tutorials\Outputs\gross2022\figures


## 3 An adaptive dashboard for manual parameters explorations 

Here the idea is to create an interactive interface to examine the previous visualizations for different sets of parameters.

Controles :
- all parameters : parameters, hyper_parameters, initial_values
- parameters_sliders [ min, max, number of steps ] If empty there is no sliders mode
        
To create  **parameters_sliders** use **encode** from basics.text_management to automatically have the list of the encoded variable names.


In [9]:
from tsma.basics.text_management import encode

print(list(encode(parameters, hyper_parameters, initial_values)))

['p1__b_margin', 'p1__hh_cons_propensity', 'p1__norisk_interest', 'p1__tgt_capital_ratio', 'p1__smooth_interest', 'p1__beta', 'p2__f_n', 'p2__hh_n', 'p2__t_end', 'p2__seed', 'p3__wages_0', 'p3__wages_1', 'p3__prices_0', 'p3__prices_1']


Then decide the ranges of the sliders

In [10]:
params_sliders = {
    "p1__b_margin": [0, 1, 100],
    "p1__hh_cons_propensity": [0, 1, 100],
    "p1__norisk_interest": [0, 1, 100],
    "p1__tgt_capital_ratio": [0, 1, 100],
    "p1__smooth_interest": [0, 100, 100],
    "p1__beta": [0, 1, 100],
    "p2__f_n": [0, 2000, 2000],
    "p2__hh_n": [0, 20000, 20000],
    "p2__t_end": [0, 10000, 10000],
    "p2__seed": [0, 2000, 2000],
    "p3__wages_0": [0, 10, 100],
    "p3__wages_1": [0, 10, 100],
    "p3__prices_0": [0, 10, 100],
    "p3__prices_1": [0, 10, 100],
}


A variable can be used as a reference to compare with **var_compared**. Here unemployement is used by default.

Finally, an app can be launch as following :


In [12]:
from tsma.visuals.dashboards import app_mosaique, run_app

if __name__ == "__main__":

     app = app_mosaique(
        parameters,
        hyper_parameters,
        initial_values,
        params_sliders,
        model,
        dct_groups,
        var_compared="sPD",
        in_jupyter=True,
    )
     run_app(app, in_jupyter = True, port = "8040")

Dash app running on http://127.0.0.1:8040/


100%|██████████| 38/38 [00:03<00:00,  9.51it/s]


### Change the Latex

With:
- **f_latex** for the latex traduction of the parameters.

- **ncol_latex** for the number of columns of parameters on the sidebar.

**f_latex** can be built by using **varname_to_latex** from  basics.text_management. \
This function  separates the components of a variable names, based on two lists : the exponents and the indexes.
Then the core of the name is converted by using core_conv dictionary. All the specific cases for indexes and exponents can be handeled with special_exp and special_index dictionaries.

In [13]:
from tsma.basics.text_management import varname_to_latex

def new_f_latex(varname):
    core_conv = {
        "n": "N",
        "margin": "\ mu",
        "cons propensity": "\ alpha",
        "norisk interest": "i",
        "capital ratio": "\mathcal{Capr}",
        "t end": "\mathcal{T}",
        "beta": "\ beta",
        "smooth interest": "\mathcal{Smooth}",
        "wages" : "\mathcal{w}"
    }
    exponents = ["hh", "f", "b", "0", "1"]
    indexes = ["tgt"]

    special_exp = {
        "margin": "b",
    }
    special_index = {
        "norisk interest": "D",
    }

    return varname_to_latex(
        varname, exponents, indexes, core_conv, special_exp, special_index
    )

print(new_f_latex("p1__tgt_capital_ratio"))

$\mathcal{Capr}_{tgt}^{}$


### Change the figure

with **f_fig** definition. \
This function has to return a plotly figure.\
To use seaborn and plt figures, you would need to treat them as images (see below).

In [14]:
from tsma.visuals.figures import mosaique

def new_f_fig(output, title, grouping, arg):
    return mosaique(output, 3, title)

Finally, a new dashboard can be defined : 

In [16]:
if __name__ == "__main__":
    app = app_mosaique(
        parameters,
        hyper_parameters,
        initial_values,
        params_sliders,
        model,
        dct_groups = {},
        in_jupyter=True,
        
        ncol_latex = 3,
        f_latex=new_f_latex,
        f_fig=new_f_fig,
    )
    run_app(app, in_jupyter = True, port = "8040")

Dash app running on http://127.0.0.1:8040/


### Add an image 
 **app_mos_and_image** show both a plotly figure and an image computed at the first update of the dashboard.
This is useful to use seaborn visualizations. However, this solution has a main issue : the cache of the app makes the update of the image impossible if the image's name doesn't change.
Thus, the function's name is used to change the saving path.

### Change the image

**f_img** save the image 
The default function computes Okun's and Phillips's curves but a sufficient number of points are needed.
Thus let us defined a new image with a custom heatmap function.


In [17]:
from tsma.visuals.figures import my_heatmap
from tsma.visuals.fig_management import save_fig
from tsma.visuals.dashboards import app_mos_and_image, run_app

def heatmap_f_img(m, save_path, img_format):
    
    df = m.output.iloc[3:, :]
    fig = my_heatmap(df, "new_img")
    fig.savefig(save_path, dpi=200, format=img_format)


if __name__ == "__main__":

    app = app_mos_and_image(
        parameters,
        hyper_parameters,
        initial_values,
        {},
        model,
        dct_groups,
        in_jupyter=True,
        f_img = heatmap_f_img
    )
    run_app(app, in_jupyter = True, port = "8040")

Dash app running on http://127.0.0.1:8040/
